In [1]:
import pandas as pd
import numpy as np
import nltk
import nltk.sentiment.util
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import pprint
import re
import gensim
import gensim.corpora as corpora
import os
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import string
import math

In [2]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [3]:
df = pd.read_csv('../../amazon_reviews_us_Grocery_v1_00_sentiment.csv').sample(500000)

df = df.drop(["marketplace", "customer_id", "product_parent", "product_category", "review_date"], axis=1)

df = df[~df['review_body'].isnull()]

df['review_body'] = df['review_headline'] + '. ' + df['review_body']

df["health_hazard"] = np.nan

df = df.set_index("review_id")

In [4]:
df

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard
review_id,,,,,,,,,,,,
RBMMWU9JUVS49,B00KO8H8IK,"Trolli Evil Twins Gummies Candy, 4.25 Ounce --...",5,1,1,N,Y,She was happy with the package,She was happy with the package. These were for...,0.200000,0.400000,NaN
RW3WW83NN1W40,B006ULT3WY,"Chimes Orange Ginger Chews, 1lb Bag",5,0,0,N,Y,Five Stars,Five Stars. Arrived on time. They do seem to h...,0.000000,0.000000,NaN
R1M7PJ6L2PFV5B,B000F0FVMW,"Bearitos Organic Microwave Popcorn, No Oil Add...",5,0,0,N,Y,Thank you!!!,Thank you!!!. Thank you to Bearitos for provid...,0.291667,0.375000,NaN
R363ZAJB9PLYJ3,B007M8FRTW,"Yogi Sweet Tangerine Positive Energy, 16 Tea Bags",5,1,1,N,Y,"Still Jet Lagged, but SMILING","Still Jet Lagged, but SMILING. Love it. Won't ...",0.400000,0.433333,NaN
R11ZNPC8UTXF4T,B0006N5DGW,Kasugai Lychee Candy 4.05oz,5,0,0,N,Y,Five Stars,Five Stars. they were great but I wish there w...,0.650000,0.625000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
R38BMXTW5NKSDX,B00CXXKE9O,Community Coffee Premium Ground Coffee,5,0,0,N,Y,Makes me miss Louisiana (but the scent brings ...,Makes me miss Louisiana (but the scent brings ...,1.000000,0.300000,NaN
RZ8JCZB8D11PK,B004GCUWAQ,Swanson Canned Meat,3,0,0,N,Y,Three Stars,Three Stars. meh,0.000000,0.000000,NaN
RJW2UL20AIKSP,B0083LJ18A,Golazo Caffeinated Electrolyte Drink,5,1,1,N,N,Something new and different in a sports drink...,Something new and different in a sports drink....,0.050505,0.603535,NaN


In [5]:
neg_reviews = df.query("star_rating < 4", engine="python")

In [6]:
stop_words = stopwords.words('english')
stop_words.extend(['br'])
translator=str.maketrans('','',string.punctuation)

#neg_reviews["tokened_review"] = np.nan
all_tokens = []
#neg_reviews.astype({"tokened_review": 'object'}).dtypes
for index, row in neg_reviews.iterrows():
    text = row["review_body"].lower()
    text = text.translate(translator)
    text = word_tokenize(text)
    new_text = []
    for token in text:
        token = stemmer.stem(token)
        if token not in stop_words:
            new_text.append(token)
    all_tokens.append(new_text)
neg_reviews["tokened_review"] = all_tokens

<ipython-input-6-eba5c76ea0a6>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_reviews["tokened_review"] = all_tokens


In [7]:
neg_reviews

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,
R3KU2WR5IO0W4X,B0096RP89G,Campbell's Go Soup Microwavable Pouch,2,0,0,N,Y,Surprised @ Campbells..UGH.,Surprised @ Campbells..UGH.. Having tried and ...,0.102326,0.510826,NaN,"[surpris, campbellsugh, tri, love, spici, chor..."
R18BKD0VMDDIFS,B00M08RY4K,Nestle Buncha Crunch Fun to Munch Minis Non-Re...,2,0,0,N,Y,Don't order chocolate from Amazon in the summer.,Don't order chocolate from Amazon in the summe...,0.150000,0.250000,NaN,"[dont, order, chocol, amazon, summer, love, ca..."
R2LEWTZD5PJOZM,B00AFJOYHQ,Lavazza Espresso Intenso for Keurig Rivo Syste...,3,1,3,N,Y,Three Stars,Three Stars. Nespresso all the way,0.000000,0.000000,NaN,"[three, star, nespresso, way]"
R21HN5VIZIIFG6,B0046HJN5G,KIND Minis Variety Pack,3,0,0,N,Y,Three Stars,Three Stars. Sticky and too sweet.,0.350000,0.650000,NaN,"[three, star, sticki, sweet]"
R1AVMHAWNTGV3F,B0000DIXKU,Alaska Smokehouse Smoked Salmon Fillet in Wood...,2,3,8,N,Y,More than advertised,More than advertised. While I agree this is sh...,0.231944,0.454167,NaN,"[advertis, agre, thi, ship, nice, wooden, box,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
R2JQIH2AWEIRH4,B00F1YN6IS,Gummi Bears 1LB (Pink Grapefruit),1,0,3,N,Y,One Star,One Star. Less than one pound.,-0.166667,0.066667,NaN,"[one, star, less, one, pound]"
R32C3PB779GZ44,B000LL0RKG,"Silk Soymilk, Very Vanilla, 8.25-Ounce Aseptic...",1,0,0,N,N,Marketing Hype,Marketing Hype. Silk's TV ads claiming their s...,0.312500,0.572115,NaN,"[market, hype, silk, tv, ad, claim, soy, milk,..."
R2VCHVYLKZG2C0,B00JFH46H2,Crystal Light Liquid Variety Drink Mix 1.62 Fl...,2,0,0,N,Y,not good for,not good for. tended to use too much....not go...,0.700000,0.600000,NaN,"[good, tend, use, muchnot, good, us]"


In [8]:
hazardous_words=["rotten", "mouldy", "moldy", "mold", "mould", "sick", "dangerous", "diarrhea", "poisoning", "stale", "contaminate",'disease', 'disgusting', 'hospital', 'poisoning', 'hazardous', 'ill', 'mold','moldy', 'mildew', 'dangerous' 'rancid', 'smelly', 'stale', 'unhealthy', 'sick','expired','expiration']
for word in hazardous_words:
    neg_reviews.loc[[stemmer.stem(word) in tokened_review for tokened_review in neg_reviews["tokened_review"]], "health_hazard"] = 1
    

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [9]:
num_labeled = len(neg_reviews.query("health_hazard == 1", engine="python"))
num_labeled

10253

In [10]:
non_hazardous = neg_reviews.query("health_hazard != 1", engine="python").sample(3*num_labeled)
for i in list(non_hazardous.index):
    neg_reviews.loc[i, "health_hazard"] = 0

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [11]:
labeled = neg_reviews.query("health_hazard == 0 or health_hazard == 1", engine="python")
reviews = labeled.loc[:, 'tokened_review'].values
y = labeled.loc[:, 'health_hazard'].values

In [12]:
pprint.pprint(reviews)

array([list(['surpris', 'campbellsugh', 'tri', 'love', 'spici', 'chorizo', 'varieti', 'go', 'soup', 'wa', 'eager', 'tri', 'anoth', 'flavor', 'pleas', 'understand', 'write', 'review', 'item', 'love', 'want', 'share', 'good', 'find', 'onli', 'rare', 'doe', 'purchas', 'inspir', 'submit', 'neg', 'review', 'thi', 'one', 'aw', 'pour', 'dispos', 'second', 'spoon', 'first', 'tastedisbelief', 'seconddisgust', 'like', 'curri', 'enjoy', 'sever', 'varietieshot', 'mild', 'green', 'yellow', 'etc', 'like', 'ingredi', 'consist', 'pour', 'bag', 'wa', 'unpleas', 'glob', 'wateri', 'stream', 'anoth', 'globth', 'final', 'bowl', 'wa', 'wateri', 'worrisom', 'look', 'sight', 'experi', 'campbel', 'soup', 'ha', 'positivetheyr', 'exactli', 'gourmet', 'qualitylol', 'good', 'basic', 'tast', 'comfort', 'food', 'childhood', 'tast', 'wa', 'like', 'curri', 'gone', 'bad', 'old', 'mayb', 'moldi', 'cheap', 'curri', 'powder', 'got', 'meand', 'ive', 'cook', 'indian', 'chines', 'etc', '30', 'year', 'problem', 'thi', 'soup',

In [13]:
for i in range(len(reviews)):
    reviews[i] = ' '.join(reviews[i])

In [14]:
reviews

array(['surpris campbellsugh tri love spici chorizo varieti go soup wa eager tri anoth flavor pleas understand write review item love want share good find onli rare doe purchas inspir submit neg review thi one aw pour dispos second spoon first tastedisbelief seconddisgust like curri enjoy sever varietieshot mild green yellow etc like ingredi consist pour bag wa unpleas glob wateri stream anoth globth final bowl wa wateri worrisom look sight experi campbel soup ha positivetheyr exactli gourmet qualitylol good basic tast comfort food childhood tast wa like curri gone bad old mayb moldi cheap curri powder got meand ive cook indian chines etc 30 year problem thi soup defi definit review call 34perfumey34 wellsorta appear offtast cant offer ani redeem qualiti hereth veggi pretti unidentifiablea chop gruel look soup slightli greasi film topbr well feel bit bad neg trust food hit dispos without ani attempt part doctor stay away lolbr tri spici chorizo kindim huge sausag fan gener blend flavor

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
matrix = TfidfVectorizer(max_features=250)
X = matrix.fit_transform(reviews).toarray()
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.08898885],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.19059753, ..., 0.        , 0.        ,
        0.        ]])

In [16]:
# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [17]:
from sklearn.naive_bayes import GaussianNB
classifier_g = GaussianNB()

from sklearn.linear_model import LogisticRegression

classifier_l = LogisticRegression()

classifier_g.fit(X_train, y_train)
classifier_l.fit(X_train, y_train)

# Predict Class
y_pred_g = classifier_g.predict(X_test)
y_pred_l = classifier_l.predict(X_test)

# Accuracy 
from sklearn.metrics import accuracy_score
accuracy_g = accuracy_score(y_test, y_pred_g)
accuracy_l = accuracy_score(y_test, y_pred_l)
print(accuracy_g)

0.9367989856627329


In [18]:
matrix.get_feature_names()

['12',
 'actual',
 'ad',
 'add',
 'almost',
 'also',
 'amazon',
 'amount',
 'ani',
 'anoth',
 'anyth',
 'arriv',
 'aw',
 'away',
 'back',
 'bad',
 'bag',
 'bar',
 'bean',
 'becaus',
 'befor',
 'best',
 'better',
 'big',
 'bit',
 'bitter',
 'bottl',
 'bought',
 'box',
 'brand',
 'brew',
 'broken',
 'butter',
 'buy',
 'call',
 'came',
 'candi',
 'cant',
 'care',
 'case',
 'chang',
 'chees',
 'chip',
 'chocol',
 'coconut',
 'coffe',
 'color',
 'come',
 'compani',
 'contain',
 'cook',
 'cooki',
 'cost',
 'could',
 'couldnt',
 'cup',
 'date',
 'day',
 'definit',
 'didnt',
 'differ',
 'disappoint',
 'disgust',
 'doe',
 'doesnt',
 'dont',
 'dri',
 'drink',
 'eat',
 'end',
 'enjoy',
 'enough',
 'even',
 'ever',
 'everi',
 'expect',
 'expens',
 'expir',
 'far',
 'favorit',
 'feel',
 'find',
 'fine',
 'first',
 'flavor',
 'food',
 'found',
 'free',
 'fresh',
 'fruit',
 'get',
 'gift',
 'give',
 'go',
 'good',
 'got',
 'great',
 'green',
 'guess',
 'ha',
 'half',
 'hard',
 'high',
 'hope',
 'horr

In [19]:
trial = neg_reviews
trial_x = trial.loc[:,"tokened_review"].values
for i in range(len(trial_x)):
    trial_x[i] = ' '.join(trial_x[i])
trial_X = matrix.fit_transform(trial_x).toarray()
trial_y = classifier_l.predict(trial_X)
trial_y

array([1., 0., 1., ..., 0., 1., 1.])

In [20]:
neg_reviews["health_hazard"] = trial_y

<ipython-input-20-56d181f868ab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_reviews["health_hazard"] = trial_y


In [21]:
neg_reviews

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,
R3KU2WR5IO0W4X,B0096RP89G,Campbell's Go Soup Microwavable Pouch,2,0,0,N,Y,Surprised @ Campbells..UGH.,Surprised @ Campbells..UGH.. Having tried and ...,0.102326,0.510826,1.0,surpris campbellsugh tri love spici chorizo va...
R18BKD0VMDDIFS,B00M08RY4K,Nestle Buncha Crunch Fun to Munch Minis Non-Re...,2,0,0,N,Y,Don't order chocolate from Amazon in the summer.,Don't order chocolate from Amazon in the summe...,0.150000,0.250000,0.0,dont order chocol amazon summer love candi got...
R2LEWTZD5PJOZM,B00AFJOYHQ,Lavazza Espresso Intenso for Keurig Rivo Syste...,3,1,3,N,Y,Three Stars,Three Stars. Nespresso all the way,0.000000,0.000000,1.0,three star nespresso way
R21HN5VIZIIFG6,B0046HJN5G,KIND Minis Variety Pack,3,0,0,N,Y,Three Stars,Three Stars. Sticky and too sweet.,0.350000,0.650000,1.0,three star sticki sweet
R1AVMHAWNTGV3F,B0000DIXKU,Alaska Smokehouse Smoked Salmon Fillet in Wood...,2,3,8,N,Y,More than advertised,More than advertised. While I agree this is sh...,0.231944,0.454167,0.0,advertis agre thi ship nice wooden box use thi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
R2JQIH2AWEIRH4,B00F1YN6IS,Gummi Bears 1LB (Pink Grapefruit),1,0,3,N,Y,One Star,One Star. Less than one pound.,-0.166667,0.066667,1.0,one star less one pound
R32C3PB779GZ44,B000LL0RKG,"Silk Soymilk, Very Vanilla, 8.25-Ounce Aseptic...",1,0,0,N,N,Marketing Hype,Marketing Hype. Silk's TV ads claiming their s...,0.312500,0.572115,0.0,market hype silk tv ad claim soy milk delici s...
R2VCHVYLKZG2C0,B00JFH46H2,Crystal Light Liquid Variety Drink Mix 1.62 Fl...,2,0,0,N,Y,not good for,not good for. tended to use too much....not go...,0.700000,0.600000,0.0,good tend use muchnot good us


In [22]:
dangerous = neg_reviews.query("health_hazard == 1", engine="python")
dangerous

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,
R3KU2WR5IO0W4X,B0096RP89G,Campbell's Go Soup Microwavable Pouch,2,0,0,N,Y,Surprised @ Campbells..UGH.,Surprised @ Campbells..UGH.. Having tried and ...,0.102326,0.510826,1.0,surpris campbellsugh tri love spici chorizo va...
R2LEWTZD5PJOZM,B00AFJOYHQ,Lavazza Espresso Intenso for Keurig Rivo Syste...,3,1,3,N,Y,Three Stars,Three Stars. Nespresso all the way,0.000000,0.000000,1.0,three star nespresso way
R21HN5VIZIIFG6,B0046HJN5G,KIND Minis Variety Pack,3,0,0,N,Y,Three Stars,Three Stars. Sticky and too sweet.,0.350000,0.650000,1.0,three star sticki sweet
R303H0C6056MZC,B00CB1QP46,Stevia powder,3,0,0,N,Y,Three Stars,Three Stars. if you use too much it has a afte...,0.200000,0.200000,1.0,three star use much ha tast
R2VM5V625SMXVC,B000N9N7I0,"Marshalls Creek Spices Lemon Pepper Seasoning,...",2,1,1,N,Y,no lemon taste,no lemon taste. There was no llist of ingredie...,-0.031250,0.293750,1.0,lemon tast wa llist ingredi thought wa requir ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
R3UAIESRFGN6BS,B001J11HR4,"Torani Sugar Free Peanut Butter Syrup, 750mL",2,4,4,N,Y,disappointing,disappointing. I had high hopes for this syrup...,-0.073889,0.346111,1.0,disappoint high hope thi syrup mix chocol diet...
R1YITXMKXYDTVC,B00BJJ327Y,Viviana Certified Gluten-Free Lemon Garlic Orz...,1,0,0,N,Y,One Star,One Star. weird flavor,-0.500000,1.000000,1.0,one star weird flavor
R2JQIH2AWEIRH4,B00F1YN6IS,Gummi Bears 1LB (Pink Grapefruit),1,0,3,N,Y,One Star,One Star. Less than one pound.,-0.166667,0.066667,1.0,one star less one pound


In [23]:
#LDA
data_filtered = dangerous['review_body'].values
pprint.pprint(data_filtered)

array(["Surprised @ Campbells..UGH.. Having tried and loved the Spicy Chorizo variety of these Go Soups, I was eager to try another flavor. Please understand I write few reviews, most being items I love and want to share a good find. Only rarely does a purchase inspire me to submit a negative review. This is one of those, it being so awful, I poured it down the disposal after the second spoonful ( first taste..disbelief, second..disgust). I like curry, have enjoyed several varieties..hot, mild, green, yellow, etc. I like all the ingredients. The consistency as it poured out of the bag was unpleasant: a glob, a watery stream, another glob....the final bowl was a watery, worrisome looking sight. But my experience with Campbells soups has been positive..they're not exactly gourmet quality,lol, but good basic tastes. Comfort foods from childhood :-)<br /><br />The taste was like curry that's gone bad, old.. maybe moldy?  Cheap curry powder? You got me..and I've cooked Indian, Chinese, etc 

In [24]:
def sent_to_words(sentences):
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = sentence.translate(translator)
        yield(nltk.word_tokenize(str(sentence)))

data_words = list(sent_to_words(data_filtered))
clean_data = []
for i in range(len(data_words)):
    clean_review = []
    for j in range(len(data_words[i])):
        if data_words[i][j] not in stop_words:
            clean_review.append(data_words[i][j])
    clean_data.append(clean_review)

In [25]:
# Create Dictionary
id2word = corpora.Dictionary(clean_data)
# Create Corpus
texts = clean_data
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
pprint.pprint(corpus[:5])

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 2),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 2),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 2),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 3),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 2),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 2),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 3),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 3),
  (77, 1),
  (78, 1),
  (79, 2),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 2),
  (89, 1),
  (90, 1),
  (91, 1)

In [26]:
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       minimum_probability=0.1,
                                       workers=3,
                                       passes=2)
# Print the Keyword in the 10 topics
pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.009*"product" + 0.008*"one" + 0.007*"good" + 0.007*"ingredients" + '
  '0.006*"doesnt" + 0.005*"like" + 0.005*"bag" + 0.005*"star" + 0.005*"would" '
  '+ 0.004*"first"'),
 (1,
  '0.026*"stars" + 0.023*"three" + 0.017*"like" + 0.016*"coffee" + 0.015*"tea" '
  '+ 0.013*"good" + 0.012*"taste" + 0.010*"flavor" + 0.009*"doesnt" + '
  '0.006*"great"'),
 (2,
  '0.017*"taste" + 0.016*"stars" + 0.016*"like" + 0.012*"two" + 0.010*"flavor" '
  '+ 0.008*"good" + 0.008*"doesnt" + 0.008*"product" + 0.007*"would" + '
  '0.007*"much"'),
 (3,
  '0.028*"one" + 0.022*"star" + 0.013*"like" + 0.011*"taste" + 0.010*"stars" + '
  '0.009*"good" + 0.008*"product" + 0.007*"would" + 0.006*"three" + '
  '0.006*"doesnt"'),
 (4,
  '0.013*"product" + 0.009*"ingredients" + 0.007*"stars" + 0.006*"sugar" + '
  '0.006*"one" + 0.006*"taste" + 0.006*"like" + 0.005*"would" + 0.005*"flavor" '
  '+ 0.005*"two"')]


In [27]:
#from itertools import chain
#lda_corpus = lda_model[corpus]

# Find the threshold, let's set the threshold to be 1/#clusters,
# To prove that the threshold is sane, we average the sum of all probabilities:
#scores = list(chain(*[[score for topic_id,score in topic] for topic in [doc for doc in lda_corpus]]))
#threshold = sum(scores)/len(scores)


In [28]:
import pickle 
import pyLDAvis
import pyLDAvis.gensim_models
data_vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

pyLDAvis.display(data_vis)

In [29]:
labelled_df = neg_reviews

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
labelled_df.drop(columns=["helpful_votes", "total_votes", "vine", "verified_purchase"])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,product_id,product_title,star_rating,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,
R3KU2WR5IO0W4X,B0096RP89G,Campbell's Go Soup Microwavable Pouch,2,Surprised @ Campbells..UGH.,Surprised @ Campbells..UGH.. Having tried and ...,0.102326,0.510826,1.0,surpris campbellsugh tri love spici chorizo va...
R18BKD0VMDDIFS,B00M08RY4K,Nestle Buncha Crunch Fun to Munch Minis Non-Re...,2,Don't order chocolate from Amazon in the summer.,Don't order chocolate from Amazon in the summe...,0.150000,0.250000,0.0,dont order chocol amazon summer love candi got...
R2LEWTZD5PJOZM,B00AFJOYHQ,Lavazza Espresso Intenso for Keurig Rivo Syste...,3,Three Stars,Three Stars. Nespresso all the way,0.000000,0.000000,1.0,three star nespresso way
R21HN5VIZIIFG6,B0046HJN5G,KIND Minis Variety Pack,3,Three Stars,Three Stars. Sticky and too sweet.,0.350000,0.650000,1.0,three star sticki sweet
R1AVMHAWNTGV3F,B0000DIXKU,Alaska Smokehouse Smoked Salmon Fillet in Wood...,2,More than advertised,More than advertised. While I agree this is sh...,0.231944,0.454167,0.0,advertis agre thi ship nice wooden box use thi...
...,...,...,...,...,...,...,...,...,...
R2JQIH2AWEIRH4,B00F1YN6IS,Gummi Bears 1LB (Pink Grapefruit),1,One Star,One Star. Less than one pound.,-0.166667,0.066667,1.0,one star less one pound
R32C3PB779GZ44,B000LL0RKG,"Silk Soymilk, Very Vanilla, 8.25-Ounce Aseptic...",1,Marketing Hype,Marketing Hype. Silk's TV ads claiming their s...,0.312500,0.572115,0.0,market hype silk tv ad claim soy milk delici s...
R2VCHVYLKZG2C0,B00JFH46H2,Crystal Light Liquid Variety Drink Mix 1.62 Fl...,2,not good for,not good for. tended to use too much....not go...,0.700000,0.600000,0.0,good tend use muchnot good us


In [32]:
labelled_df.to_csv('labelled_df.csv')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
